<a href="https://colab.research.google.com/github/valesierrai/Introduccion-a-la-Inteligencia-de-Negocios/blob/main/Subconsultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Subconsultas

##0. Importamos Librerias

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1. Cargamos la Base de Datos

In [ ]:
#Nos conectamos con la base de datos
conexion = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/IB/MisCancionesV3.db')
#Creamos el cursor
cursor = conexion.cursor()

##Subconsulta Autónomas
En este caso la información que requiere la consulta principal proviene toda de la misma tabla, aunque la subconsulta requiere de tablas adicionales

In [ ]:
#¿Cuáles son los títulos, la duración, y el idioma de todas las canciones del género "Salsa"?

consulta = '''
           SELECT título,
           duración,
           idioma
           FROM Canciones
           WHERE idGénero = (SELECT idGénero
                              FROM Géneros
                              WHERE nombre = 'Salsa')
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Una aventura', '00:05:16', 'español')
('Gotas de lluvia', '00:05:54', 'español')
('Ne me quitte pas', '00:05:37', 'francés')
('Cali es sabrosura', '00:03:14', 'español')


In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tablas = cursor.fetchall()
for tabla in tablas:
  print(tabla)

('Géneros',)
('Álbumes',)
('Usuarios',)
('Intérpretes',)
('Canciones',)
('Calificaciones',)
('Reproducciones',)
('ListasReproducción',)
('CancionesLista',)


In [ ]:
#¿Cuáles son los títulos, la duración y el idioma de la canciones del género Salsa cuyo intérprete principal es Yuri Buenaventura?

consulta = '''
           SELECT título,
           duración,
           idioma
           FROM Canciones
           WHERE idGénero = (SELECT idGénero
                              FROM Géneros
                              WHERE nombre = 'Salsa') AND
                 idIntérpretePrincipal = (SELECT idIntérprete
                                          FROM intérpretes
                                          WHERE nombre = 'Yuri Buenaventura')
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Ne me quitte pas', '00:05:37', 'francés')
('Cali es sabrosura', '00:03:14', 'español')


In [ ]:
#¿Cuál es la duración de la canción más larga
consulta = '''
           SELECT MAX(duración)
           FROM Canciones
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('00:05:54',)


In [ ]:
#¿Cuál es la duración de la canción más corta
consulta = '''
           SELECT MIN(duración)
           FROM Canciones
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('00:02:28',)


In [ ]:
#¿Cuál es el título y duración de las canciones más largas y las canciones más cortas?
#En este caso se hará la consulta principal y la subconsulta de la misma tabla

consulta = '''
           SELECT título,
           duración
           FROM Canciones
           WHERE duración IN ((SELECT MIN(duración)
                               FROM Canciones),
                              (SELECT MAX(duración)
                               FROM Canciones))
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Gotas de lluvia', '00:05:54')
('Oropel', '00:02:28')


In [ ]:
#¿Cuál es el título y la duración de las canciones interpretadas por solistas?
#Se hace primero sin subconsulta
#1. Seleccionamos el ID de los intérpretes que son solistas
consulta = '''
           SELECT idIntérprete
           FROM Intérpretes
           WHERE tipoIntérprete = 'Solista'
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

(1,)
(3,)
(5,)
(7,)
(9,)


In [ ]:
#2. Seleccionamos o mostramos el título, la duración de las canciones cuyo intérprete corresponda al ID encontrado en la consulta anterior

consulta = '''
           SELECT título,
           duración
           FROM Canciones
           WHERE idIntérpretePrincipal IN (1, 3, 5, 7, 9)
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('La tierra del olvido', '00:04:25')
('La bicicleta', '00:03:47')
('Hips dont lie', '00:03:38')
('Ojos así', '00:03:55')
('Amarte mas no pude', '00:04:49')
('Sin medir distancias', '00:04:58')
('Mi gente', '00:03:09')
('Ginza', '00:02:51')
('Ne me quitte pas', '00:05:37')
('Cali es sabrosura', '00:03:14')


In [ ]:
#Ahora, se hace en formato de subconsulta

consulta = '''
           SELECT título,
           duración
           FROM Canciones
           WHERE idIntérpretePrincipal IN (SELECT idIntérprete
                                           FROM Intérpretes
                                           WHERE tipoIntérprete = 'Solista')
          ORDER BY título
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Amarte mas no pude', '00:04:49')
('Cali es sabrosura', '00:03:14')
('Ginza', '00:02:51')
('Hips dont lie', '00:03:38')
('La bicicleta', '00:03:47')
('La tierra del olvido', '00:04:25')
('Mi gente', '00:03:09')
('Ne me quitte pas', '00:05:37')
('Ojos así', '00:03:55')
('Sin medir distancias', '00:04:58')


In [ ]:
#¿Cuál es el título de las canciones reproducidas por mujeres nacidas en Colombia?

consulta = '''
           SELECT título
           FROM Canciones
           WHERE idCanción IN (SELECT idCanción
                               FROM Reproducciones
                               WHERE idUsuario IN (SELECT idUsuario
                                                   FROM Usuarios
                                                   WHERE sexo = 'F' AND
                                                         paísNacimiento = 'Colombia'))
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('La tierra del olvido',)
('La bicicleta',)
('Una aventura',)
('Gotas de lluvia',)
('Hips dont lie',)
('Ojos así',)
('Bolero falaz',)
('Maligno',)
('Amarte mas no pude',)
('Sin medir distancias',)
('De donde vengo yo',)
('Mi gente',)
('Ginza',)
('Las acacias',)
('Ne me quitte pas',)
('Cali es sabrosura',)
('La creciente',)
('Olvídala',)


In [ ]:
#¿Cuál es el título de las canciones del género Salsa reproducidas por mujeres en Colombia?
#¿Cuál es el título de las canciones reproducidas por mujeres nacidas en Colombia?

consulta = '''
           SELECT título
           FROM Canciones
           WHERE idCanción IN (SELECT idCanción
                               FROM Reproducciones
                               WHERE idUsuario IN (SELECT idUsuario
                                                   FROM Usuarios
                                                   WHERE sexo = 'F' AND
                                                         paísNacimiento = 'Colombia'))
                 AND idGénero = (SELECT idGénero
                                FROM Géneros
                                WHERE nombre = 'Salsa')
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Una aventura',)
('Gotas de lluvia',)
('Ne me quitte pas',)
('Cali es sabrosura',)


##Subconsultas Correlacionadas

En este caso la información que requiere la consulta principal proviene de diferentes tablas o de la misma tabla a partir de agrupaciones.

In [ ]:
#¿Cuál es el identificador del género, el título y la duración de las canciones de mayor duración de cada género?

consulta = '''
           SELECT idGénero,
                  título,
                  duración
           FROM Canciones AS TablaPrincipal
           WHERE duración = (SELECT MAX(duración)
                             FROM Canciones AS TablaSubconsulta
                             WHERE TablaSubconsulta.idGénero = TablaPrincipal.idGénero)
          '''

#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

(9, 'La tierra del olvido', '00:04:25')
(3, 'Gotas de lluvia', '00:05:54')
(1, 'Ojos así', '00:03:55')
(2, 'Maligno', '00:04:09')
(5, 'Mi gente', '00:03:09')
(6, 'Las acacias', '00:04:04')
(4, 'Olvídala', '00:05:04')


In [ ]:
#¿Cuáles son los títulos de la canciones intérpretadas por Carlos Vives que han sido agregadas a listas de reproducción y los nombres de dichas listas?

#Primero saqué el titulo de la lista de reproducción donde el id corresponda a la lista de reproducciones (Atributo 1)
#Relacioné el idCanción con el de las CancionesLista(Atributo 2)
#Por último hacemos las condiciones del intérprete principal
consulta = '''
           SELECT (SELECT título
                   FROM ListasReproducción
                   WHERE ListasReproducción.idListaReproducción = CancionesLista.idListaReproducción) AS títulolista,

                  (SELECT título
                   FROM Canciones
                   WHERE Canciones.idCanción = CancionesLista.idCanción)
                   FROM CancionesLista

                   WHERE idCanción IN (SELECT idCanción
                                       FROM Canciones
                                       WHERE idIntérpretePrincipal = (SELECT idIntérprete
                                                                     FROM Intérpretes
                                                                     WHERE nombre = 'Carlos Vives'))
          '''
#Ejecutamos la consulta
cursor = cursor.execute(consulta)

#Traemos los resultados
resultados = cursor.fetchall()

#Visualizamos
for resultado in resultados:
  print(resultado)

('Música para planchar', 'La tierra del olvido')
('Música para planchar', 'La bicicleta')
('Mis favoritas', 'La bicicleta')
('Canciones para dedicar a mi novia cuando tenga novia', 'La bicicleta')
('Mi cuerpo pide salsa', 'La tierra del olvido')
('Mi cuerpo pide salsa', 'La bicicleta')
('Música para estudiar', 'La tierra del olvido')
('Música para estudiar', 'La bicicleta')
('Entrenar con energia ', 'La tierra del olvido')
('Entrenar con energia ', 'La bicicleta')
('Tarde de relajación', 'La tierra del olvido')
('Tarde de relajación', 'La bicicleta')
('Manejar con gusto, sabor y ritmo', 'La bicicleta')
('Para gustos los colores', 'La tierra del olvido')
('Reggaeton viejo', 'La tierra del olvido')
('Reggaeton viejo', 'La bicicleta')
('Música para trabajar', 'La tierra del olvido')
('Colección de imperdibles', 'La tierra del olvido')
('Crossover de ayer y hoy', 'La tierra del olvido')
('Paseo en familia', 'La tierra del olvido')
('Las melodias que inspiran', 'La tierra del olvido')
('Las